# Walmart Sales Analysis — Google Colab Notebook
هذا الملف جاهز لرفعه أو لصقه في Google Colab. **قبل التشغيل**: ارفع ملف `Walmart.csv` باستخدام واجهة رفع الملفات في Colab (أو استخدم Google Drive).

كل خلية تحتوي على شرح بالعربي يشرح ماذا يفعل الكود.

---

## 1) استيراد المكتبات المطلوبة
**شرح:** هذه المكتبات ستستخدم لمعالجة البيانات، الرسم، وبناء النموذج.

In [ ]:
# استيراد المكتبات
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

print('libraries loaded')

## 2) رفع وقراءة الملف
**شرح:** شغّل هذه الخلية لرفع `Walmart.csv` من جهازك إلى بيئة Colab ثم قراءة الملف.

(بدلاً من الرفع يمكنك وضع الملف في Google Drive وربطه إذا أردت.)

In [ ]:
# رفع ملف CSV من جهازك (سيطلب منك اختيار الملف)
from google.colab import files
uploaded = files.upload()

# بعد الرفع، اقرأ الملف
import io, os
if 'Walmart.csv' in uploaded:
    df = pd.read_csv(io.BytesIO(uploaded['Walmart.csv']))
else:
    # حاول العثور على أي ملف CSV مرفوع
    csv_files = [k for k in uploaded.keys() if k.lower().endswith('.csv')]
    if len(csv_files) > 0:
        df = pd.read_csv(io.BytesIO(uploaded[csv_files[0]]))
    else:
        raise FileNotFoundError('لم يتم العثور على Walmart.csv. ارفع الملف ثم شغّل الخلية مرة أخرى.')

print('Loaded rows:', len(df))
df.head()

## 3) تحويل عمود التاريخ وتنظيف البيانات
**شرح:** نحول عمود Date إلى نوع تواريخ، نحذف الصفوف الناقصة في الأعمدة الأساسية ونحول الأعمدة الرقمية لصيغ رقمية ونملأ المفقودات بالوسيط.

In [ ]:
# تحويل التاريخ وتنظيف القيم المفقودة
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

# حذف أي صف لا يحتوي على تاريخ أو مبيعات
df = df.dropna(subset=['Date', 'Weekly_Sales']).copy()

# تحويل الأعمدة الرقمية والتعامل مع missing
num_cols = ['Weekly_Sales','Temperature','Fuel_Price','CPI','Unemployment','Holiday_Flag']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# ملء القيم المفقودة بالوسيط للأعمدة العددية (إن وجدت)
for c in ['Temperature','Fuel_Price','CPI','Unemployment']:
    if c in df.columns:
        df[c].fillna(df[c].median(), inplace=True)

print('After cleaning: rows=', len(df))
df.head()

## 4) إنشاء ميزات زمنية
**شرح:** نستخرج Year, Month, WeekOfYear من عمود التاريخ لاستخدامها كنماذج.

In [ ]:
# إنشاء ميزات زمنية
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

df[['Date','Year','Month','WeekOfYear']].head()

## 5) EDA — نظرة عامة
**شرح:** عرض إحصائيات وصفية ومعلومات عن الأعمدة.

In [ ]:
# نظرة عامة
display(df.head())
display(df.describe(include='all'))
print('\nInfo:')
df.info()

## 6) EDA — توزيع المبيعات
**شرح:** نرسم histogram لتوزيع Weekly_Sales.

In [ ]:
# رسم توزيع Weekly_Sales
plt.figure(figsize=(10,5))
plt.hist(df['Weekly_Sales'], bins=50)
plt.title('Distribution of Weekly Sales')
plt.xlabel('Weekly Sales')
plt.ylabel('Frequency')
plt.show()

## 7) تحليل الارتباط (Correlation)
**شرح:** نحسب مصفوفة الارتباط بين المتغيرات الرقمية المهمة.

In [ ]:
# مصفوفة الارتباط
numeric_cols = ['Weekly_Sales','Temperature','Fuel_Price','CPI','Unemployment','Holiday_Flag','Year','Month','WeekOfYear']
present_cols = [c for c in numeric_cols if c in df.columns]
corr = df[present_cols].corr()
corr

## 8) تجهيز بيانات النموذج
**شرح:** نحدد المتغيرات المستقلة X والمتغير الهدف y، ونحول Store إلى دوال 0/1 (one-hot).

In [ ]:
# تجهيز X و y
features = ['Store','Holiday_Flag','Temperature','Fuel_Price','CPI','Unemployment','Year','Month','WeekOfYear']
features = [f for f in features if f in df.columns]

X = df[features].copy()
y = df['Weekly_Sales'].copy()

# One-hot encode Store (إن وجد)
if 'Store' in X.columns:
    X = pd.get_dummies(X, columns=['Store'], prefix='Store', drop_first=True)

print('X shape:', X.shape)
X.head()

## 9) Standardize الأرقام و تدريب نموذج RandomForest
**شرح:** نطبق StandardScaler على الميزات العددية ثم ندرب RandomForest كـ baseline.

In [ ]:
# Standardize numeric features
num_feats = ['Temperature','Fuel_Price','CPI','Unemployment','Year','Month','WeekOfYear']
num_feats = [c for c in num_feats if c in X.columns]

scaler = StandardScaler()
if len(num_feats) > 0:
    X.loc[:, num_feats] = scaler.fit_transform(X[num_feats])

# تدريب RandomForest
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X, y)

# توقع على نفس البيانات (baseline quick check)
y_pred = model.predict(X)
rmse = mean_squared_error(y, y_pred, squared=False)
r2 = r2_score(y, y_pred)
print(f'RMSE (train): {rmse:.2f}')
print(f'R2 (train): {r2:.4f}')

## 10) أهمية المتغيرات (Feature Importance)
**شرح:** نستخرج أهمية المتغيرات من نموذج RandomForest ونعرض أهم 20 متغيراً.

In [ ]:
# Feature importance
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
display(importances.head(20))

# رسم أهم المتغيرات
plt.figure(figsize=(10,6))
importances.head(20).plot(kind='barh')
plt.gca().invert_yaxis()
plt.title('Top 20 Feature Importances (RandomForest)')
plt.show()

## 11) حفظ النتائج والملفات (اختياري)
**شرح:** لتحميل النتائج إلى جهازك، نفعل ZIP أو نحفظ ملفات منفردة. شغّل الخلية التالية إذا أردت حفظ artifact.

In [ ]:
# حفظ ملفات النتائج داخل مجلد 'walmart_colab_output'
import os, joblib, csv
out_dir = 'walmart_colab_output'
os.makedirs(out_dir, exist_ok=True)

# save artifacts
df.head().to_csv(os.path.join(out_dir, 'head.csv'), index=False)
df.describe().to_csv(os.path.join(out_dir, 'describe.csv'))
corr.to_csv(os.path.join(out_dir, 'correlation_matrix.csv'))
importances.head(50).to_csv(os.path.join(out_dir, 'feature_importances.csv'))
joblib.dump(model, os.path.join(out_dir, 'random_forest_model.joblib'))
print('Saved artifacts to', out_dir)

## انتهى — ملاحظات أخيرة
- هذا Notebook جاهز للتشغيل في Google Colab.
- تأكد أن ترفع `Walmart.csv` قبل تشغيل الخلايا.
- إذا تريد: أضيف خلية تقسم البيانات زمنياً لtrain/test وتقييم أفضل، أو أدرج GridSearch لتحسين الهايبرباراميترز.

**لو تحب أعمل تعديل أو أضيف جزء، قلّي أي إضافة تريد.**